<a href="https://colab.research.google.com/github/ismoil27/computer_vision/blob/main/menu_detector_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Menu Detector!')

Menu Detector!
